In [14]:
import numpy as np
import llm_blender
blender = llm_blender.Blender()
blender.loadranker("llm-blender/PairRM", device="cuda") # load PairRM

c:\Users\jonas\anaconda3\envs\cp_rank\Lib\site-packages\dataclasses_json\core.py:201: RuntimeWarning: 'NoneType' object value of non-optional type load_checkpoint detected when decoding RankerConfig.
  warnings.warn(
c:\Users\jonas\anaconda3\envs\cp_rank\Lib\site-packages\dataclasses_json\core.py:201: RuntimeWarning: 'NoneType' object value of non-optional type device detected when decoding RankerConfig.
  warnings.warn(
c:\Users\jonas\anaconda3\envs\cp_rank\Lib\site-packages\transformers\convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Successfully loaded ranker from  C:\Users\jonas\.cache\huggingface\hub\llm-blender\PairRM


In [15]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset, concatenate_datasets

# Load SNLI dataset
ds = load_dataset("ag_news").shuffle(seed=42)
full_dataset = concatenate_datasets([ds["train"], ds["test"]])


In [16]:
full_dataset
ds_cal = ds["test"]
ds_test = ds["train"]


In [17]:
dataset = full_dataset

In [13]:
test_dataset = ds_test.select(range(10))  # Get training subset
cal_dataset = ds_test.select(range(10))    # Get test subset

labels_cal = ds_cal["label"]
labels_test = ds_test["label"]

classes = ["World", "Sports", "Business", "Science and Technology"]


def compute_cal_scores(ds, classes):
    inputs = [f"{x['text']}" for x in ds]
    candidates_texts = [[f"The category is {x}" for x in classes]]*len(inputs)
    return blender.rank(inputs, candidates_texts, return_scores=True, batch_size=2**5)


cal_scores1 = compute_cal_scores(cal_dataset, classes)

Ranking candidates: 100%|██████████| 1/1 [00:01<00:00,  1.10s/it]


In [ ]:
# from sklearn.metrics import accuracy_score
# from sklearn.model_selection import StratifiedKFold
# classes = ["World", "Sports", "Business", "Science and Technology"]

# # Extract labels
# y = dataset['label']  # Keeping it in Hugging Face format

# # Define K-Fold Cross-Validation
# k_folds = 5  # Change as needed
# skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)

# def compute_cal_scores(ds):
#     inputs = [f"{x['text']}" for x in ds]
#     candidates_texts = [[f"The category is {x}" for x in classes]]*len(inputs)
#     return blender.rank(inputs, candidates_texts, return_scores=True, batch_size=2**5)


# # Perform cross-validation
# for fold, (test_idx, cal_idx) in enumerate(skf.split(range(len(dataset)), y)):
#     print(f"Fold: {fold}\n")
#     test_dataset = dataset.select(test_idx)  # Get training subset
#     cal_dataset = dataset.select(cal_idx)    # Get test subset

#     labels_cal = [x["label"] for x in cal_dataset]
#     labels_test = [x["label"] for x in test_dataset]

#     cal_scores = compute_cal_scores(cal_dataset)
#     # take scores of true labels
#     cal_scores = cal_scores[np.arange(cal_scores.shape[0]), labels_cal]
#     pred_scores = compute_cal_scores(test_dataset)


#     alphas = [0.02, 0.05, 0.1, 0.2]
#     for alpha in alphas:
#         print("\n\n")
#         print(f"alpha =\t\t\t {alpha}")
#         n = len(cal_scores)
#         threshold = np.quantile(cal_scores.flatten(), np.ceil((n+1)*(alpha))/n, method="inverted_cdf")
#         pred_sets = [np.where(row > threshold)[0].tolist() for row in pred_scores]
#         predictions = np.argmax(pred_scores, axis=1)
#         coverage = np.mean([labels_test[i] in pred_sets[i] for i in range(n)])
#         avg_set_size = np.mean([len(s) for s in pred_sets])
#         median_set_size = np.median([len(s) for s in pred_sets])
#         accuracy = accuracy_score(labels_test, predictions)
#         print(f"coverage =\t\t {coverage}")
#         print(f"mean set size =\t\t {avg_set_size}")
#         print(f"median set size =\t {median_set_size}")
#         print(f"accuracy =\t {accuracy}")

In [19]:
print(cal_scores1)

[[ 2.3809814  -0.65008545  0.41375732 -2.1446533 ]
 [-1.276123    2.058838    0.09277344 -0.8754883 ]
 [ 1.5783691  -1.5183105  -0.04516602 -0.01489258]
 [-1.8623047   0.3505249   0.6983032   0.81347656]
 [ 0.35327148 -0.6035156  -0.00610352  0.25634766]
 [-2.2354736  -1.9589233  -1.6195679   5.813965  ]
 [ 2.144043   -0.81781006 -0.8018799  -0.524353  ]
 [ 0.90527344 -4.4975586   3.2286377   0.36364746]
 [-0.8607178  -1.3920898   1.0412598   1.2115479 ]
 [-1.383667    0.8964844   1.53479    -1.0476074 ]]


In [22]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
classes = ["World", "Sports", "Business", "Science and Technology"]

def compute_cal_scores(ds):
    inputs = [f"{x['text']}" for x in ds]
    candidates_texts = [[f"The category is {x}" for x in classes]]*len(inputs)
    return blender.rank(inputs, candidates_texts, return_scores=True, batch_size=2**5)

test_dataset = ds_test  # Get training subset
cal_dataset = ds_cal    # Get test subset

labels_cal = ds_cal["label"]
labels_test = ds_test["label"]

cal_scores = compute_cal_scores(cal_dataset)
# take scores of true labels
cal_scores = cal_scores[np.arange(cal_scores.shape[0]), labels_cal]
pred_scores = compute_cal_scores(test_dataset)


alphas = [0.02, 0.05, 0.1, 0.2]
for alpha in alphas:
    print("\n\n")
    print(f"alpha =\t\t\t {alpha}")
    n = len(cal_scores)
    threshold = np.quantile(cal_scores.flatten(), np.ceil((n+1)*(alpha))/n, method="inverted_cdf")
    pred_sets = [np.where(row > threshold)[0].tolist() for row in pred_scores]
    predictions = np.argmax(pred_scores, axis=1)
    coverage = np.mean([labels_test[i] in pred_sets[i] for i in range(n)])
    avg_set_size = np.mean([len(s) for s in pred_sets])
    median_set_size = np.median([len(s) for s in pred_sets])
    accuracy = accuracy_score(labels_test, predictions)
    print(f"coverage =\t\t {coverage}")
    print(f"mean set size =\t\t {avg_set_size}")
    print(f"median set size =\t {median_set_size}")
    print(f"accuracy =\t {accuracy}")



Ranking candidates:   1%|          | 2/238 [00:04<08:24,  2.14s/it]


KeyboardInterrupt: 